In [175]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime
import pandas as pd

# Set the limit for number of articles to download
LIMIT = 5000

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

count = 1

# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                newsPaper['articles'].append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 100 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 100:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            newsPaper['articles'].append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

# Finally it saves the articles as a JSON-file.
try:
    with open('scraped_articles.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)


Building site for  yahoo finance
1 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/video/eurasia-group-impeachment-may-cause-203414770.html
2 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/micron-earnings-what-to-know-in-markets-thursday-000457651.html
3 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/trump-impeachment-inquiry-could-kill-the-usmca-233627048.html
4 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/one-way-trump-could-send-stocks-soaring-233628338.html
5 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/blackberry-plunges-below-devilish-6-135130404.html
6 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/fairfax-watsa-lost-121-million-164238512.html
7 articles downloaded from yahoo finance  using newspaper, ur

55 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/fitbit-versa-2-review-130030290.html
56 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/sonos-ceo-investors-are-underestimating-the-power-of-our-new-speakers-152630312.html
57 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/sonos-move-portable-speaker-130025107.html
58 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/slacks-first-quarterly-earnings-report-after-ipo-190935065.html
59 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/news/roku-smart-soundbar-subwoofer-speaker-130147197.html
60 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/calendar/earnings?day=2019-09-26
61 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/calendar/splits?

112 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/video/author-actress-jill-kargman-money-200000592.html
113 articles downloaded from yahoo finance  using newspaper, url:  https://finance.yahoo.com/video/trumps-risky-tax-hike-213310868.html
1 articles downloaded from google , url:  https://www.nytimes.com/2019/09/25/us/politics/trump-ukraine-whistleblower.html
2 articles downloaded from google , url:  https://www.cnn.com/2019/09/25/politics/mike-quigley-trump-whistleblower-complaint-cnntv/index.html
3 articles downloaded from google , url:  https://www.politico.com/news/2019/09/25/mcconnell-ukraine-trump-001034
4 articles downloaded from google , url:  https://www.npr.org/2019/09/25/764453663/pentagon-letter-undercuts-trump-assertion-on-delaying-aid-to-ukraine-over-corrup
5 articles downloaded from google , url:  https://www.politico.com/story/2019/09/25/trump-pence-ukraine-phone-calls-1512771
6 articles downloaded from google , url:  https://

15 articles downloaded from theguardian , url:  https://www.theguardian.com/us-news/2019/sep/25/stanford-sexual-assault-victim-chanel-miller-interview
16 articles downloaded from theguardian , url:  https://www.theguardian.com/stage/2019/sep/26/glass-kill-bluebeard-imp-review-finding-fascination-in-bloodshot-fables
17 articles downloaded from theguardian , url:  https://www.theguardian.com/politics/2019/sep/25/incredible-sulks-anger-is-followed-by-ranting-of-geoffrey-cox
18 articles downloaded from theguardian , url:  https://www.theguardian.com/politics/2019/sep/25/labour-media-tactics-more-trumpian-admit-jeremy-corbyn
19 articles downloaded from theguardian , url:  https://www.theguardian.com/stage/2019/sep/25/two-ladies-review-zoe-wanamaker-bridge-theatre-london-nancy-harris-nicholas-hytner
20 articles downloaded from theguardian , url:  https://www.theguardian.com/politics/audio/2019/sep/26/can-labour-unite-and-plot-a-path-to-power
21 articles downloaded from theguardian , url:  ht

67 articles downloaded from theguardian , url:  https://www.theguardian.com/music/2019/sep/25/bowie-gnome-and-dr-dre-hospital-musicians-embarrassing-early-songs
68 articles downloaded from theguardian , url:  https://www.theguardian.com/film/2019/sep/25/to-tokyo-review-caspar-seale-jones-florence-kosky
69 articles downloaded from theguardian , url:  https://www.theguardian.com/music/2019/sep/25/louis-tomlinson-one-direction-dark-side-gives-me-strength
70 articles downloaded from theguardian , url:  https://www.theguardian.com/live-victoriously/2019/sep/20/early-afternoon-delight-the-boundless-appeal-of-brunching
71 articles downloaded from theguardian , url:  https://www.theguardian.com/live-victoriously/2019/sep/17/neil-rankin-if-you-keep-your-food-and-drink-simple-the-night-is-more-sociable
72 articles downloaded from theguardian , url:  https://www.theguardian.com/live-victoriously/2019/sep/18/jessie-ware-i-feel-much-more-comfortable-in-my-30s-im-living-victoriously
73 articles down

...

16 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/war-on-reality-intensifies-transgender-man-upset-to-not-be-listed-as-father-of-child-she-birthed/
17 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/war-on-reality-intensifies-transgender-man-upset-to-not-be-listed-as-father-of-child-she-birthed/#vuukle-comments
18 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/alex-jones-sues-young-turks-official-statement/
19 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/alex-jones-sues-young-turks-official-statement/#vuukle-comments
20 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/msm-deception-watch-dishonest-pundits-blatantly-lie-about-trump-ukraine-transcript/
21 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/msm-deception-watch-dishonest-pundits-blatantly-lie-about-trump-ukraine-tran

68 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/watch-live-dems-drink-impeachment-kool-aid/#vuukle-comments
69 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/can-beto-come-and-take-it/
70 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/can-beto-come-and-take-it/#vuukle-comments
71 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/not-so-shocking-liberals-triggered-at-chicago-anti-trump-rally/
72 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/not-so-shocking-liberals-triggered-at-chicago-anti-trump-rally/#vuukle-comments
73 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/pueblo-sin-fronteras-united-methodist-church-in-chicago/
74 articles downloaded from infowars  using newspaper, url:  https://www.infowars.com/pueblo-sin-fronteras-united-methodist-church-in-chicago/#vuukle-com

38  Article has date of type None...
39  Article has date of type None...
40  Article has date of type None...
41  Article has date of type None...
42  Article has date of type None...
43  Article has date of type None...
44  Article has date of type None...
45  Article has date of type None...
46  Article has date of type None...
47  Article has date of type None...
48  Article has date of type None...
49  Article has date of type None...
50  Article has date of type None...
51  Article has date of type None...
52  Article has date of type None...
53  Article has date of type None...
54  Article has date of type None...
55  Article has date of type None...
56  Article has date of type None...
57  Article has date of type None...
58  Article has date of type None...
59  Article has date of type None...
60  Article has date of type None...
61  Article has date of type None...
62  Article has date of type None...
63  Article has date of type None...
64  Article has date of type None...
6

4  Article has date of type None...
5  Article has date of type None...
6  Article has date of type None...
7  Article has date of type None...
8  Article has date of type None...
9  Article has date of type None...
10  Article has date of type None...
11  Article has date of type None...
Building site for  rsschomp
1  Article has date of type None...
2  Article has date of type None...
3  Article has date of type None...
4 articles downloaded from rsschomp  using newspaper, url:  https://commonthought.net/what-is-the-difference-between-tired-and-sleepy/?utm_source=rss&utm_medium=rss&utm_campaign=what-is-the-difference-between-tired-and-sleepy
5  Article has date of type None...
6  Article has date of type None...
Building site for  realtyfeed
1  Article has date of type None...
2  Article has date of type None...
3  Article has date of type None...
4  Article has date of type None...
5  Article has date of type None...
Building site for  feedsee
Building site for  philstockworld
1  Ar

Article `download()` failed with 404 Client Error: Not Found for url: https://www.thestreet.com/_yahoo/video/strategysession/10438485.html?cm_ven=YAHOOV&cm_cat=FREE&cm_ite=NA#10438485 on URL http://www.thestreet.com/_yahoo/video/strategysession/10438485.html?cm_ven=YAHOOV&cm_cat=FREE&cm_ite=NA#10438485
continuing...
41 articles downloaded from philstockworld  using newspaper, url:  http://www.fool.com/investing/high-growth/2008/09/19/amds-fusion-confusion.aspx
Article `download()` failed with 403 Client Error: Forbidden for url: https://seekingalpha.com/article/96355-the-next-smart-green-grid-tech-wimax?source=yahoo on URL http://seekingalpha.com/article/96355-the-next-smart-green-grid-tech-wimax?source=yahoo
continuing...
42  Article has date of type None...
43 articles downloaded from philstockworld  using newspaper, url:  http://www.philstockworld.com/2009/08/28/educational-videos/
44 articles downloaded from philstockworld  using newspaper, url:  http://www.philstockworld.com/2010/

In [228]:
import pandas as pd
df=pd.DataFrame(columns=["STOCK_ID", "URL", "DATE_OF_PUBLISH", "HEADLINE", "CONTENT"])
content=[]
stock=[]
url=[]
dop=[]
title=[]
# with open("scraped_articles.json",'r') as load_f:
#       data4 = json.load(load_f)
newspapers=data["newspapers"]
# print(newspapers)
articles=newspapers["theguardian"]
#print (articles)

for articles in newspapers:



    news=newspapers[articles]
#     print (d)
#     print (d["articles"])

    if news["articles"]!= []:
        e=news['articles']
#         print(e)
        for i in range(len(e)):
            j=e[i]
            
            content.append(j['text'])
            title.append(j['title'])
            url.append(j['link'])
            dop.append(j['published'])
            
df["URL"]=url
df["DATE_OF_PUBLISH"]=dop
df["HEADLINE"]=title
df["CONTENT"]=content
df.head(100)
df.drop_duplicates(inplace=True)
df.to_csv("C:/data/CS/CS_NEWSARTICLE_SCRAPE.csv")